In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
market_data = pd.read_csv("./data/Bitcoin Price Weekly 2022 to 2010 .csv")
graph_parameters= pd.read_csv("./data/weeklyparameters.csv")

In [5]:
graph_parameters

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.143607,-0.070265,0.278090,0.215956,0.163888,-0.468321,-0.134060,-0.053859,-0.105646,0.122768,...,0.362808,0.368988,0.364313,0.352094,0.355708,0.344235,0.353294,0.373881,0.358176,0.355856
1,0.089774,-0.207905,0.256683,0.040967,0.301744,-0.479524,-0.059471,0.023795,-0.089099,0.026119,...,0.344774,0.349917,0.348386,0.340110,0.360732,0.347816,0.350748,0.354506,0.354432,0.346472
2,0.086321,-0.093323,0.228172,0.157205,0.196785,-0.458902,-0.220363,-0.102736,-0.017346,-0.027238,...,0.340600,0.362379,0.346005,0.337340,0.351533,0.349064,0.345180,0.353104,0.349887,0.348563
3,-0.004186,-0.129215,0.149419,0.147874,0.170400,-0.376984,-0.022998,0.089453,-0.143320,-0.039410,...,0.298482,0.305089,0.290398,0.289368,0.307518,0.313389,0.308761,0.307860,0.285055,0.292567
4,0.047130,-0.083402,0.200965,0.053718,0.051699,-0.309155,-0.093535,0.001527,-0.036476,0.015799,...,0.254703,0.275766,0.265648,0.259244,0.257194,0.265005,0.275387,0.269150,0.264928,0.272635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0.075948,-0.207777,0.200187,0.108073,0.168021,-0.274515,-0.053430,0.007518,-0.049993,0.062387,...,0.300513,0.336553,0.308040,0.314696,0.325846,0.325187,0.308636,0.318336,0.318691,0.318112
82,0.110836,-0.230830,0.263283,0.199076,0.203319,-0.308236,-0.082092,-0.010337,-0.073649,0.047009,...,0.307592,0.314960,0.313105,0.313973,0.327114,0.308763,0.310107,0.323382,0.315543,0.326845
83,0.057832,-0.274646,0.268859,0.112171,0.144329,-0.258795,-0.100073,0.070667,-0.101681,0.019881,...,0.309662,0.323282,0.315687,0.311634,0.314007,0.328940,0.310238,0.324160,0.329759,0.323687
84,-0.013597,-0.175680,0.256521,0.143769,0.160563,-0.230204,-0.067204,0.066503,-0.092091,0.081259,...,0.325128,0.317551,0.311203,0.313594,0.310596,0.333461,0.308023,0.317215,0.326639,0.313724


In [6]:
6/7/2011

0.0004262271790864531

In [7]:
full_df= pd.concat([market_data.iloc[50:50+graph_parameters.shape[0],1:].reset_index().drop('index',axis=1),graph_parameters],axis=1).iloc[:87]

In [8]:
full_df

,Price,Open,High,Low,Vol.,Change %,0,1,2,3,...,246,247,248,249,250,251,252,253,254,255
0,14.4,15.4,16.5,11.0,366200.0,-6.62,0.143607,-0.070265,0.278090,0.215956,...,0.362808,0.368988,0.364313,0.352094,0.355708,0.344235,0.353294,0.373881,0.358176,0.355856
1,13.7,14.4,15.7,13.5,190880.0,-4.59,0.089774,-0.207905,0.256683,0.040967,...,0.344774,0.349917,0.348386,0.340110,0.360732,0.347816,0.350748,0.354506,0.354432,0.346472
2,13.7,13.7,14.7,12.5,263670.0,0.00,0.086321,-0.093323,0.228172,0.157205,...,0.340600,0.362379,0.346005,0.337340,0.351533,0.349064,0.345180,0.353104,0.349887,0.348563
3,13.5,13.7,14.7,13.3,132600.0,-1.10,-0.004186,-0.129215,0.149419,0.147874,...,0.298482,0.305089,0.290398,0.289368,0.307518,0.313389,0.308761,0.307860,0.285055,0.292567
4,6.6,13.5,14.9,5.7,398270.0,-51.59,0.047130,-0.083402,0.200965,0.053718,...,0.254703,0.275766,0.265648,0.259244,0.257194,0.265005,0.275387,0.269150,0.264928,0.272635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,17.9,15.6,19.2,15.4,485490.0,14.47,0.075948,-0.207777,0.200187,0.108073,...,0.300513,0.336553,0.308040,0.314696,0.325846,0.325187,0.308636,0.318336,0.318691,0.318112
82,19.6,17.9,21.4,17.2,442230.0,9.79,0.110836,-0.230830,0.263283,0.199076,...,0.307592,0.314960,0.313105,0.313973,0.327114,0.308763,0.310107,0.323382,0.315543,0.326845
83,23.6,19.6,24.0,19.4,318710.0,20.48,0.057832,-0.274646,0.268859,0.112171,...,0.309662,0.323282,0.315687,0.311634,0.314007,0.328940,0.310238,0.324160,0.329759,0.323687
84,27.2,23.6,27.6,21.7,466620.0,15.10,-0.013597,-0.175680,0.256521,0.143769,...,0.325128,0.317551,0.311203,0.313594,0.310596,0.333461,0.308023,0.317215,0.326639,0.313724


In [9]:
x = full_df

In [10]:
Y = []
threshold=3
for i in market_data["Change %"][51:51+graph_parameters.shape[0]]:
    if abs(i)>threshold:
        if i>0:
            Y.append(1)
        else:
            Y.append(-1)
    else:
        Y.append(0)

In [11]:
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(scaled_x, Y, test_size=0.33, random_state=42)

svm_model           = svm.SVC()
stockhastic_model   = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
knn_model           = KNeighborsClassifier(n_neighbors=5)
decisiontree_model  = DecisionTreeClassifier()
random_forest_model = RandomForestClassifier(max_depth=5, random_state=0)

svm_model.fit(x_train, y_train)
stockhastic_model.fit(x_train, y_train)
knn_model.fit(x_train, y_train)
decisiontree_model.fit(x_train, y_train)
random_forest_model.fit(x_train, y_train)

prediction_svm_model          = svm_model.predict(x_test)
prediction_stockhastic_model  = stockhastic_model.predict(x_test)
prediction_knn_model          = knn_model.predict(x_test)
prediction_decisiontree_model = decisiontree_model.predict(x_test)
prediction_random_forest_model= random_forest_model.predict(x_test)

f1_score_svm_model           = f1_score(y_test, prediction_svm_model,          average='weighted')
f1_score_stockhastic_model   = f1_score(y_test, prediction_stockhastic_model,  average='weighted')
f1_score_knn_model           = f1_score(y_test, prediction_knn_model,          average='weighted')
f1_score_decisiontree_model  = f1_score(y_test, prediction_decisiontree_model, average='weighted')
f1_score_random_forest_model = f1_score(y_test, prediction_random_forest_model,average="weighted")

precision_score_svm_model           = precision_score(y_test, prediction_svm_model          , average='weighted')
precision_score_stockhastic_model   = precision_score(y_test, prediction_stockhastic_model  , average='weighted')
precision_score_knn_model           = precision_score(y_test, prediction_knn_model          , average='weighted')
precision_score_decisiontree_model  = precision_score(y_test, prediction_decisiontree_model , average='weighted')
precision_score_random_forest_model= precision_score(y_test, prediction_random_forest_model, average="weighted")

accuracy_score_svm_model           = accuracy_score(y_test, prediction_svm_model          )
accuracy_score_stockhastic_model   = accuracy_score(y_test, prediction_stockhastic_model  )
accuracy_score_knn_model           = accuracy_score(y_test, prediction_knn_model          )
accuracy_score_decisiontree_model  = accuracy_score(y_test, prediction_decisiontree_model )
accuracy_score_random_forest_model = accuracy_score(y_test, prediction_random_forest_model)

scores= [[f1_score_svm_model, precision_score_svm_model, accuracy_score_svm_model]\
        ,[f1_score_stockhastic_model, precision_score_stockhastic_model, accuracy_score_stockhastic_model]\
        ,[f1_score_knn_model, precision_score_knn_model, accuracy_score_knn_model]\
        ,[f1_score_decisiontree_model, precision_score_decisiontree_model, accuracy_score_decisiontree_model]\
        ,[f1_score_random_forest_model,precision_score_random_forest_model, accuracy_score_random_forest_model]]
temp_df = pd.DataFrame(scores)
temp_df.columns = ["F1", "Precision", "Accuracy"]
temp_df = temp_df.T
temp_df.columns=["SVM", "Stockhastic", "KNN", "DecisionTree", "Random Forest"]
results = (temp_df.T)

In [12]:
results

,F1,Precision,Accuracy
SVM,0.262405,0.192118,0.413793
Stockhastic,0.392889,0.399631,0.413793
KNN,0.376026,0.377586,0.379310
DecisionTree,0.249042,0.273399,0.275862
Random Forest,0.303553,0.294253,0.344828


In [13]:
x = market_data.iloc[50:50+graph_parameters.shape[0],1:]

In [14]:
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(scaled_x, Y, test_size=0.33, random_state=42)

svm_model           = svm.SVC()
stockhastic_model   = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
knn_model           = KNeighborsClassifier(n_neighbors=5)
decisiontree_model  = DecisionTreeClassifier()
random_forest_model = RandomForestClassifier(max_depth=5, random_state=0)

svm_model.fit(x_train, y_train)
stockhastic_model.fit(x_train, y_train)
knn_model.fit(x_train, y_train)
decisiontree_model.fit(x_train, y_train)
random_forest_model.fit(x_train, y_train)

prediction_svm_model          = svm_model.predict(x_test)
prediction_stockhastic_model  = stockhastic_model.predict(x_test)
prediction_knn_model          = knn_model.predict(x_test)
prediction_decisiontree_model = decisiontree_model.predict(x_test)
prediction_random_forest_model= random_forest_model.predict(x_test)

f1_score_svm_model           = f1_score(y_test, prediction_svm_model,          average='weighted')
f1_score_stockhastic_model   = f1_score(y_test, prediction_stockhastic_model,  average='weighted')
f1_score_knn_model           = f1_score(y_test, prediction_knn_model,          average='weighted')
f1_score_decisiontree_model  = f1_score(y_test, prediction_decisiontree_model, average='weighted')
f1_score_random_forest_model = f1_score(y_test, prediction_random_forest_model,average="weighted")

precision_score_svm_model           = precision_score(y_test, prediction_svm_model          , average='weighted')
precision_score_stockhastic_model   = precision_score(y_test, prediction_stockhastic_model  , average='weighted')
precision_score_knn_model           = precision_score(y_test, prediction_knn_model          , average='weighted')
precision_score_decisiontree_model  = precision_score(y_test, prediction_decisiontree_model , average='weighted')
precision_score_random_forest_model= precision_score(y_test, prediction_random_forest_model, average="weighted")

accuracy_score_svm_model           = accuracy_score(y_test, prediction_svm_model          )
accuracy_score_stockhastic_model   = accuracy_score(y_test, prediction_stockhastic_model  )
accuracy_score_knn_model           = accuracy_score(y_test, prediction_knn_model          )
accuracy_score_decisiontree_model  = accuracy_score(y_test, prediction_decisiontree_model )
accuracy_score_random_forest_model = accuracy_score(y_test, prediction_random_forest_model)

scores= [[f1_score_svm_model, precision_score_svm_model, accuracy_score_svm_model]\
        ,[f1_score_stockhastic_model, precision_score_stockhastic_model, accuracy_score_stockhastic_model]\
        ,[f1_score_knn_model, precision_score_knn_model, accuracy_score_knn_model]\
        ,[f1_score_decisiontree_model, precision_score_decisiontree_model, accuracy_score_decisiontree_model]\
        ,[f1_score_random_forest_model,precision_score_random_forest_model, accuracy_score_random_forest_model]]
temp_df = pd.DataFrame(scores)
temp_df.columns = ["F1", "Precision", "Accuracy"]
temp_df = temp_df.T
temp_df.columns=["SVM", "Stockhastic", "KNN", "DecisionTree", "Random Forest"]
results = (temp_df.T)

In [15]:
results

,F1,Precision,Accuracy
SVM,0.331034,0.279919,0.413793
Stockhastic,0.345040,0.346470,0.344828
KNN,0.294436,0.345813,0.310345
DecisionTree,0.450308,0.505040,0.448276
Random Forest,0.406571,0.429258,0.448276
